# Topic Modeling for 20Newsgroups

### Latent Dirichlet Allocation

dataset: http://qwone.com/~jason/20Newsgroups/

#### Data Retrival

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_colwidth', -1)

In [5]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
type(newsgroups_train)

sklearn.utils.Bunch

In [6]:
from pprint import pprint

pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [106]:
train = fetch_20newsgroups(subset='train', categories=['sci.crypt', 
                                                       'sci.electronics', 
                                                       'sci.med',
                                                       'sci.space',
                                                       'soc.religion.christian',
                                                       'alt.atheism'
                                                      ])
train.target_names

['alt.atheism',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian']

In [107]:
train_df = pd.DataFrame(train.data)
train_df.columns = ['text']
train_df['label'] = train.target

In [108]:
train_df.head()

,text,label
0,"From: gerald.belton@ozonehole.com (Gerald Belton) \nSubject: Food Dehydrators\nDistribution: world\nOrganization: Ozone Online Operations, Inc. - New Orleans, LA\nReply-To: gerald.belton@ozonehole.com (Gerald Belton) \nLines: 39\n\n>Does anybody out there have one of those food dehydrators I've been seeing\n>all over late-night TV recently? I was wondering if they use forced air, hea\n>or both. If there's heat involved, anybody know what temperature they run at\n>My wife would like one and I'm not inclined to pay >$100.00 for a box, a fan\n>and a heater. Seems to me you should be able to throw a dehydrator together\n>for just a few bucks. Heck, the technology is only what? 1,000 years old?\n\nYou can learn how to build a deyhdrator very easily from the book, ""The\nHungry Hiker's Guide to Good Food,"" by Gretchen McHugh. The heat source\nis a 100 watt light bulb. Basically, it's a vertical wooden box with\nventilation holes in the top and bottom (lots of them, you want the air\nto flow). The light bulb goes in the bottom, and wire cake racks are\nspaced every 6"" starting about 10"" above the bulb.\n\nOr, at a slightly higher cost in electricity, you can do what I do: Use\nyour oven. (NOTE - I do this in an electric oven; some gas ovens may\nnot have a low enough setting). Put food to be dried on cookie tins or\nracks in the oven. Set oven to 140 degrees (the lowest setting on my\noven - if yours goes down to 120 that's probably even better.) Stick a\nwooden spoon or something across the front corner of the oven and close\nthe door on the spoon so that it stays open about an inch - this allows\nfor airflow. Leave the stuff in the oven for 6 to 8 hours; check it\noften, since this dries it much faster than the dehydrator. If you are\nusing cookie sheets instead of racks, turn the stuff over halfway\nthrough.\n\nIf you want more info, e-mail me since this isn't really the right sub\nfor this stuff.\n\ngerald.belton@ozonehole.com\n\n\n * SLMR 2.1 * I still miss my boss, but my aim is improving.\n \n----\nThe Ozone Hole BBS * A Private Bulletin Board Service * (504)891-3142\n3 Full Service Nodes * USRobotics 16.8K bps * 10 Gigs * 100,000 Files\nSKYDIVE New Orleans! * RIME Network Mail HUB * 500+ Usenet Newsgroups\nPlease route all questions or inquiries to: postmaster@ozonehole.com\n",2
1,"From: Donald Mackie <Donald_Mackie@med.umich.edu>\nSubject: Re: REQUEST: Gyro (souvlaki) sauce\nOrganization: UM Anesthesiology\nLines: 11\nDistribution: world\nNNTP-Posting-Host: 141.214.86.38\nX-UserAgent: Nuntius v1.1.1d9\nX-XXDate: Fri, 23 Apr 93 14:56:04 GMT\n\nIn article <1993Apr22.205341.172965@locus.com> Michael Trofimoff,\ntron@fafnir.la.locus.com writes:\n>Would anyone out there in 'net-land' happen to have an\n>authentic, sure-fire way of making this great sauce that\n>is used to adorn Gyro's and Souvlaki?\n\nI'm not sure of the exact recipe, but I'm sure acidophilus is one of\nthe major ingredients. :-)\n\nDon Mackie - his opinions\nUM Anesthesiology will disavow\n",3
2,"From: bdunn@cco.caltech.edu (Brendan Dunn)\nSubject: Re: Amusing atheists and agnostics\nOrganization: California Institute of Technology, Pasadena\nLines: 8\nNNTP-Posting-Host: punisher.caltech.edu\n\nThanks to whoever posted this wonderful parody of people who post without \nreading the FAQ! I was laughing for a good 5 minutes. Were there any \nparts of the FAQ that weren't mentioned? I think there might have been one\nor two...\n\nPlease don't tell me this wasn't a joke. I'm not ready to hear that yet...\n\nBrendan\n",0
3,"From: guncer@enuxha.eas.asu.edu (Selim Guncer )\nSubject: Re: Islam & Dress Code for women\nOrganization: Arizona State University, Tempe, AZ\nLines: 53\n\nIn article <16BA7103C3.I3150101@dbstu1.rz.tu-bs.de> I3150101@dbstu1.rz.tu-bs.de (Benedikt Rosenau) writes:\n>In article <1993Apr5.091258.11830@monu6.cc.monash.edu.au>\n>darice@yoyo.cc.monash.edu.au (Fred Rice) writes:\n> \n>>(2) Do women have souls in Islam?\n>>\n>>People have 

#### Data Pre-processing
- tokenization : 
  - split the text into sentences and then sentences into words
  - lower case words
  - remove punctuation
  - remove all stopwords
- Lemmatization : 
 - verbs in past and future tenses are changed into present
- Stemmazation :
 - words are reduced to their root form

In [109]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import strip_numeric
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [110]:
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/grace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [111]:
import re
train_df.text = train_df.text.map(lambda x: re.sub('^From:[ ()a-zA-Z.,\n]*Subject:', '', x))

#remove emails
train_df.text = train_df.text.map(lambda x: re.sub('\S*@\S*\s?', '', x))
train_df.head()

,text,label
0,"From: (Gerald Belton) \nSubject: Food Dehydrators\nDistribution: world\nOrganization: Ozone Online Operations, Inc. - New Orleans, LA\nReply-To: (Gerald Belton) \nLines: 39\n\n>Does anybody out there have one of those food dehydrators I've been seeing\n>all over late-night TV recently? I was wondering if they use forced air, hea\n>or both. If there's heat involved, anybody know what temperature they run at\n>My wife would like one and I'm not inclined to pay >$100.00 for a box, a fan\n>and a heater. Seems to me you should be able to throw a dehydrator together\n>for just a few bucks. Heck, the technology is only what? 1,000 years old?\n\nYou can learn how to build a deyhdrator very easily from the book, ""The\nHungry Hiker's Guide to Good Food,"" by Gretchen McHugh. The heat source\nis a 100 watt light bulb. Basically, it's a vertical wooden box with\nventilation holes in the top and bottom (lots of them, you want the air\nto flow). The light bulb goes in the bottom, and wire cake racks are\nspaced every 6"" starting about 10"" above the bulb.\n\nOr, at a slightly higher cost in electricity, you can do what I do: Use\nyour oven. (NOTE - I do this in an electric oven; some gas ovens may\nnot have a low enough setting). Put food to be dried on cookie tins or\nracks in the oven. Set oven to 140 degrees (the lowest setting on my\noven - if yours goes down to 120 that's probably even better.) Stick a\nwooden spoon or something across the front corner of the oven and close\nthe door on the spoon so that it stays open about an inch - this allows\nfor airflow. Leave the stuff in the oven for 6 to 8 hours; check it\noften, since this dries it much faster than the dehydrator. If you are\nusing cookie sheets instead of racks, turn the stuff over halfway\nthrough.\n\nIf you want more info, e-mail me since this isn't really the right sub\nfor this stuff.\n\n\n\n * SLMR 2.1 * I still miss my boss, but my aim is improving.\n \n----\nThe Ozone Hole BBS * A Private Bulletin Board Service * (504)891-3142\n3 Full Service Nodes * USRobotics 16.8K bps * 10 Gigs * 100,000 Files\nSKYDIVE New Orleans! * RIME Network Mail HUB * 500+ Usenet Newsgroups\nPlease route all questions or inquiries to:",2
1,"From: Donald Mackie Subject: Re: REQUEST: Gyro (souvlaki) sauce\nOrganization: UM Anesthesiology\nLines: 11\nDistribution: world\nNNTP-Posting-Host: 141.214.86.38\nX-UserAgent: Nuntius v1.1.1d9\nX-XXDate: Fri, 23 Apr 93 14:56:04 GMT\n\nIn article Michael Trofimoff,\nwrites:\n>Would anyone out there in 'net-land' happen to have an\n>authentic, sure-fire way of making this great sauce that\n>is used to adorn Gyro's and Souvlaki?\n\nI'm not sure of the exact recipe, but I'm sure acidophilus is one of\nthe major ingredients. :-)\n\nDon Mackie - his opinions\nUM Anesthesiology will disavow\n",3
2,"From: (Brendan Dunn)\nSubject: Re: Amusing atheists and agnostics\nOrganization: California Institute of Technology, Pasadena\nLines: 8\nNNTP-Posting-Host: punisher.caltech.edu\n\nThanks to whoever posted this wonderful parody of people who post without \nreading the FAQ! I was laughing for a good 5 minutes. Were there any \nparts of the FAQ that weren't mentioned? I think there might have been one\nor two...\n\nPlease don't tell me this wasn't a joke. I'm not ready to hear that yet...\n\nBrendan\n",0
3,"From: (Selim Guncer )\nSubject: Re: Islam & Dress Code for women\nOrganization: Arizona State University, Tempe, AZ\nLines: 53\n\nIn article (Benedikt Rosenau) writes:\n>In article (Fred Rice) writes:\n> \n>>(2) Do women have souls in Islam?\n>>\n>>People have said here that some Muslims say that women do not have\n>>souls. I must admit I have never heard of such a view being held by\n>>Muslims of any era. I have heard of some Christians of some eras\n>>holding this viewpoint, but not Muslims. Are you sure you might not be\n>>confusing Christian history with Islamic history?\n>>\n> \n>Yes, it is supposed to have been a predominant view in the Turkish\n

In [112]:
stemmer = SnowballStemmer('english')
STOP_WORDS = list(gensim.parsing.preprocessing.STOPWORDS)

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text, deacc=True): #deacc = removes punctuations
        if token not in STOP_WORDS and len(token)>2:
            result.append(lemmatize_stemming(token))
    return result#' '.join(result)

In [113]:
%%time
#process the text, save the results as processed_docs
processed_docs = pd.DataFrame()
processed_docs = pd.concat([train_df, train_df.text.map(preprocess)], axis=1)
processed_docs.columns = ['text', 'label', 'vect']
processed_docs['text_vect'] = processed_docs.vect.map(lambda x: ' '.join(x))

CPU times: user 13.4 s, sys: 4 ms, total: 13.4 s
Wall time: 13.4 s


In [114]:
processed_docs[:5]

,text,label,vect,text_vect
0,"From: (Gerald Belton) \nSubject: Food Dehydrators\nDistribution: world\nOrganization: Ozone Online Operations, Inc. - New Orleans, LA\nReply-To: (Gerald Belton) \nLines: 39\n\n>Does anybody out there have one of those food dehydrators I've been seeing\n>all over late-night TV recently? I was wondering if they use forced air, hea\n>or both. If there's heat involved, anybody know what temperature they run at\n>My wife would like one and I'm not inclined to pay >$100.00 for a box, a fan\n>and a heater. Seems to me you should be able to throw a dehydrator together\n>for just a few bucks. Heck, the technology is only what? 1,000 years old?\n\nYou can learn how to build a deyhdrator very easily from the book, ""The\nHungry Hiker's Guide to Good Food,"" by Gretchen McHugh. The heat source\nis a 100 watt light bulb. Basically, it's a vertical wooden box with\nventilation holes in the top and bottom (lots of them, you want the air\nto flow). The light bulb goes in the bottom, and wire cake racks are\nspaced every 6"" starting about 10"" above the bulb.\n\nOr, at a slightly higher cost in electricity, you can do what I do: Use\nyour oven. (NOTE - I do this in an electric oven; some gas ovens may\nnot have a low enough setting). Put food to be dried on cookie tins or\nracks in the oven. Set oven to 140 degrees (the lowest setting on my\noven - if yours goes down to 120 that's probably even better.) Stick a\nwooden spoon or something across the front corner of the oven and close\nthe door on the spoon so that it stays open about an inch - this allows\nfor airflow. Leave the stuff in the oven for 6 to 8 hours; check it\noften, since this dries it much faster than the dehydrator. If you are\nusing cookie sheets instead of racks, turn the stuff over halfway\nthrough.\n\nIf you want more info, e-mail me since this isn't really the right sub\nfor this stuff.\n\n\n\n * SLMR 2.1 * I still miss my boss, but my aim is improving.\n \n----\nThe Ozone Hole BBS * A Private Bulletin Board Service * (504)891-3142\n3 Full Service Nodes * USRobotics 16.8K bps * 10 Gigs * 100,000 Files\nSKYDIVE New Orleans! * RIME Network Mail HUB * 500+ Usenet Newsgroups\nPlease route all questions or inquiries to:",2,"[gerald, belton, subject, food, dehydr, distribut, world, organ, ozon, onlin, oper, new, orlean, repli, gerald, belton, line, anybodi, food, dehydr, see, late, night, recent, wonder, use, forc, air, hea, heat, involv, anybodi, know, temperatur, run, wife, like, inclin, pay, box, fan, heater, abl, throw, dehydr, buck, heck, technolog, year, old, learn, build, deyhdrat, easili, book, hungri, hiker, guid, good, food, gretchen, mchugh, heat, sourc, watt, light, bulb, basic, vertic, wooden, box, ventil, hole, lot, want, air, flow, light, bulb, go, wire, cake, rack, space, start, bulb, slight, higher, cost, electr, use, oven, note, electr, oven, gas, oven, low, set, food, ...]",gerald belton subject food dehydr distribut world organ ozon onlin oper new orlean repli gerald belton line anybodi food dehydr see late night recent wonder use forc air hea heat involv anybodi know temperatur run wife like inclin pay box fan heater abl throw dehydr buck heck technolog year old learn build deyhdrat easili book hungri hiker guid good food gretchen mchugh heat sourc watt light bulb basic vertic wooden box ventil hole lot want air flow light bulb go wire cake rack space start bulb slight higher cost electr use oven note electr oven gas oven low set food dri cooki tin rack oven set oven degre lowest set oven go probabl better stick wooden spoon corner oven close door spoon stay open inch allow airflow leav stuff oven hour check dri faster dehydr cooki sheet instead rack turn stuff halfway want info mail isn right sub stuff slmr miss boss aim improv ozon hole bbs privat bulletin board servic servic nod usrobot bps gig file skydiv new orlean rime network mail hub usenet newsgroup rout question inquiri
1,"From: Donald Mackie Subject: Re: REQUEST: Gyr

### bag of words on the data set
- create a dictionary from 'processed_docs' containing the number of times a word appears in the training set

In [181]:
dictionary_made_by = processed_docs.vect
dictionary_made_by_str = 'unigram'

dictionary = gensim.corpora.Dictionary(dictionary_made_by)
print('size of the dictionary %d' %len(dictionary))
dictionary.filter_extremes(no_below=0.1, no_above=0.8)#, keep_n=dict_size)
dictionary.compactify()
print('size of the dictionary %d' %len(dictionary))
max_dic_size = len(dictionary)
dict_size = max_dic_size
# dict_size = [6000]

bow_corpus = [dictionary.doc2bow(doc) for doc in dictionary_made_by]

size of the dictionary 24695
size of the dictionary 24692


#### TF-IDF
- tf-idf model on bow_corpus

- tf = count(word, document) / len(document)
- idf = log( len(collection) / count(document_containing_term, collection)
- tf-idf = tf * idf

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=dict_size, stop_words='english')
vectors = vectorizer.fit_transform(processed_docs.text_vect)
vectors_features = vectorizer.get_feature_names()
vectors.shape

(3452, 24611)

#### Optimal number of topics
- build many LDA models with different values of number of topics and pick the one that gives the highest coherence value based on corpus
- 토픽 모델링 결과로 나온 주제들에 대해 각각의 주제에서 상위 N개의 단어를 뽑습니다. 모델링이 잘 되었을수록 한 주제 안에는 의미론적으로 유사한 단어가 많이 모여있게 마련입니다. 따라서 상위 단어 간의 유사도를 계산하면 실제로 해당 주제가 의미론적으로 일치하는 단어들끼리 모여있는지 알 수 있습니다.

In [117]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
num_topics = [i for i in range(2, 10)]
for n in num_topics:
    lda = LatentDirichletAllocation(n_topics=n, max_iter=10, learning_decay=0.7,
                                    learning_method='online', learning_offset=10.,random_state=0).fit(vectors)
    
    display_topics(lda, vectors_features, no_top_words)
    
    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda.score(vectors))
    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda.perplexity(vectors))


/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
trinomi mov dirk sedat sturm oxaprozin daypro junghann van gilligan
Topic 1:
write god line subject organ articl know post key peopl
Log Likelihood:  -238127.07703714422
Perplexity:  15403.781113643492


/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
oliveira egalon claudio trinomi mov tahiti dirk keesler junghann gilligan
Topic 1:
write god line subject organ articl know post key peopl
Topic 2:
fujitsu choler smd daypro oxaprozin sturm hfe vce kiria dopler
Log Likelihood:  -241419.74212465156
Perplexity:  17600.74714455263


/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


...

Topic 0:
venido traer paxil reprehensit gop catechist tenei vulgata mateo pensar
Topic 1:
foreskin choler speculum eyepiec moloney geoffm gia dopler kiria ebcdic
Topic 2:
write god line subject organ articl know post key peopl
Topic 3:
trinomi oxaprozin daypro anello tia bloodcount lindgren scand mtv thermocoupl
Topic 4:
egalon oliveira claudio ovul bidder putz tahiti hunchback dirk keesler
Topic 5:
burzynski claar knbr landi hfe seta ryukoku vce antineoplaston faber
Topic 6:
arcad mov lige hap hensley nextwork tra omg ecc sma
Log Likelihood:  -250791.05088513353
Perplexity:  25723.613218327275


/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
putz rolla clator blockley umc murdoch umr cpk dickhead butler
Topic 1:
key space chip encrypt line clipper organ subject post use
Topic 2:
trinomi scand het een landau jess irreduc slit dorsal davenport
Topic 3:
god christian write peopl think know articl believ subject line
Topic 4:
claar knbr dirk grino andr junghann realidad brandt fishman hprtnyc
Topic 5:
fujitsu smd idc powel hdd rainer malzbend microdisk scsi fyzzick
Topic 6:
ovul ebcdic neurotic fujita fertilz mccrae melenchol costa extravers mbti
Topic 7:
fpu souvlaki vasomotor gyro rhiniti calgari speculum keesler daypro oxaprozin
Log Likelihood:  -255343.6343743048
Perplexity:  30930.878987895918


/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
tron squarewav malzbend rainer fyzzick fettron fetron newest zation hpfcmgw
Topic 1:
write god line subject organ articl know post key peopl
Topic 2:
phobo computrac sedat tranquil ebcdic sacramento pix het deimo larouch
Topic 3:
claar knbr keesler sturm mtv loftus stansberi kubriek faber hprtnyc
Topic 4:
fujitsu trinomi landi smd dirk idc junghann lindgren scand powel
Topic 5:
gehrel jupit brader radii perijov softquad hfe inconst peri vce
Topic 6:
souvlaki ovul choler paxil hunchback oxaprozin daypro adorn trofimoff sacrum
Topic 7:
seta ryukoku nigel debugg sarchoidosi deseas bournemouth hoi vacant glaucoma
Topic 8:
putz lige grino rolla oven realidad hensley nextwork sma ecc
Log Likelihood:  -254280.50834594318
Perplexity:  29627.59943352082


In [120]:
#GridSearch

search_params = {'n_components': [i for i in range(4, 10)], 'learning_decay': [.5, .7, .9]}
lda = LatentDirichletAllocation(n_jobs=4)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(vectors)

/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in t

/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in t

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=8,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_components': [4, 5, 6, 7, 8, 9], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [121]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(vectors))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 4}
Best Log Likelihood Score:  -89217.57893561426
Model Perplexity:  19655.11673157945


In [125]:
# fasttext- out of vocabulary, spelling mistake처리에 용이

In [146]:
from gensim.models import FastText
from gensim.test.utils import common_texts
# from gensim.models import KeyedVectors

print(common_texts[0])
['human', 'interface', 'computer']

# model = FastText(size=len(processed_docs.text_vect.tolist()), window=3, min_count=1)  # instantiate
# model.build_vocab(sentences=processed_docs.text_vect.tolist())
# model.train(sentences=processed_docs.text_vect.tolist(), total_examples=len(processed_docs.text_vect.tolist()), epochs=10)

['human', 'interface', 'computer']


['human', 'interface', 'computer']

In [176]:
import gensim.downloader as api
from gensim.models import Word2Vec

#https://github.com/RaRe-Technologies/gensim-data
# word_vectors = api.load("glove-wiki-gigaword-300")
word_vectors = api.load("fasttext-wiki-news-subwords-300")

# model = Word2Vec(processed_docs.text_vect.tolist(), size=10000, window=5, min_count=1, workers=4)

[======================----------------------------] 45.2% 433.2/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[========================--------------------------] 48.2% 462.1/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========================-------------------------] 51.4% 492.7/958.4MB downloaded

...

[==============================================----] 93.3% 893.9/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================================================--] 96.6% 925.7/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 958.5/958.4MB downloaded


In [183]:
# from fastText import load_model
from itertools import chain
from collections import Counter
from numpy.linalg import eig

word_appearance = Counter(chain.from_iterable(processed_docs.vect))
word_df = pd.DataFrame(list(word_appearance.items()), columns=['word', 'occurence'])
word_df['p_occurence']  = word_df.occurence / word_df.occurence.sum()
p_dict = {r['word']: r['p_occurence'] for _, r in word_df.iterrows()}

def get_sentence_vector(ls, a=10e-4):
    """Average word vector for given list of words using fastText

    ref: A Simple but Tough-to-Beat Baseline for Sentence Embeddings, ICLR 2017
    https://openreview.net/pdf?id=SyK00v5xx
    """
    wvs = []
    for w in ls:
        try:
            wv = word_vectors.wv[w]
#             wv = model.wv[w]
        except KeyError:
            wv = 0
        if w is not None:
            wvs.append([wv, p_dict[w]])
    wvs = np.array(wvs)
    return np.mean(wvs[:, 0] * (a / (a + wvs[:, 1])), axis=0)

X = np.vstack(processed_docs.vect.map(get_sentence_vector))
eig_val, eig_vec = eig(X.T.dot(X))
u = eig_vec[:, 0].reshape(-1, 1).T
X_sent = np.vstack([x - u.T.dot(u).dot(x) for x in X])
y = pd.get_dummies(processed_docs.label).as_matrix()

In [188]:
min_neg = np.amin(X_sent)
X_sent = (-1*min_neg) + X_sent

In [192]:
np.min(X_sent)

0.0

In [193]:
from scipy import sparse

sparseX = sparse.csr_matrix(X_sent)
sparseX

# from sklearn.preprocessing import normalize
# x_normalized = normalize(sparseX, norm='l2', axis=1)

<3452x300 sparse matrix of type '<class 'numpy.float32'>'
	with 1035599 stored elements in Compressed Sparse Row format>

In [195]:
#GridSearch

search_params = {'n_components': [i for i in range(4, 10)], 'learning_decay': [.5, .7, .9]}
lda = LatentDirichletAllocation(n_jobs=4)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(sparseX)

/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in t

/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/home/grace/.local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in t

GridSearchCV(cv=None, error_score='raise',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=4,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_components': [4, 5, 6, 7, 8, 9], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [198]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(sparseX))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 4}
Best Log Likelihood Score:  -134743.4106375915
Model Perplexity:  364.45463437772906


### Find dominant topic
- find the most contributed topic

In [49]:
processed_docs = processed_docs.merge(documents, on='id')
# test = processed_docs[['id', 'title_x', 'bow_corpus', 'title_y']]
# test.to_csv('test.csv')

In [50]:
processed_docs.shape

(32049, 5)

In [51]:
processed_docs.head(5)

,id,title_x,bow_corpus,index,title_y
0,4106,"[analysi, efficaci]","[(0, 1), (1, 1)]",0,Analysis of efficacy
1,4107,"[comparison, postop, ca, level, surviv, espac, conoko, jaspac, trial]","[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]",1,Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials
2,4108,"[pattern, diseas, relaps]","[(8, 1), (9, 1), (10, 1)]",2,Pattern of disease relapse
3,4109,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]","[(11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]",3,Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine
4,4112,"[treatment, zoledron, acid]","[(16, 1), (17, 1), (18, 1)]",4,Treatment with zoledronic acid


In [52]:
processed_docs.iloc[151]

id            4325                                     
title_x       [current, patient, status, follow]       
bow_corpus    [(101, 1), (178, 1), (262, 1), (456, 1)] 
index         151                                      
title_y       Current Patient Status at Last Follow-up 
Name: 151, dtype: object

In [53]:
import random

def format_topics_sentences(model, \
                            model_type='mallet', \
                            corpus=processed_docs, \
                            texts=dictionary_made_by):
    sent_topic_df = pd.DataFrame()
    if model_type=='tfidf':
        target_corpus = corpus.tfidf_corpus
    else:
        target_corpus = corpus.bow_corpus
    
    for i, row in enumerate(model[target_corpus]):
        origin_info = processed_docs.iloc[i]
        text_vec = texts[i]
        #get main topic in each document
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j==0: #dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
#                 print(pd.Series([origin_info.id,\
#                                 int(topic_num), \
#                                 round(prop_topic, 4), \
#                                 topic_keywords, \
#                                origin_info.title_y, \
#                                text_vec]))
                sent_topic_df = sent_topic_df.append(pd.Series([origin_info.id,\
                                                                int(topic_num), \
                                                                round(prop_topic, 4), \
                                                                topic_keywords, \
                                                               origin_info.title_y, \
                                                               text_vec]), ignore_index=True)
            else:
                break
    sent_topic_df.columns = ['id', 'Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Origin_Text', 'Text_Vec']
    
    return (sent_topic_df)

In [54]:
df_topic_sents_keywords_mallet = format_topics_sentences(model=mallet_model, \
                                                  corpus=processed_docs, \
                                                  texts=dictionary_made_by, \
                                                 model_type='mallet')
df_topic_sents_keywords_mallet.head(max_coherence_topic_num)

,id,Dominant_Topic,Perc_Contribution,Topic_Keywords,Origin_Text,Text_Vec
0,4106,1.0,0.1394,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Analysis of efficacy,"[analysi, efficaci]"
1,4107,2.0,0.1473,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,"[comparison, postop, ca, level, surviv, espac, conoko, jaspac, trial]"
2,4108,0.0,0.1368,"patient, data, comparison, score, respons, time, accord, test, status, assess",Pattern of disease relapse,"[pattern, diseas, relaps]"
3,4109,1.0,0.1898,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]"
4,4112,2.0,0.1515,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Treatment with zoledronic acid,"[treatment, zoledron, acid]"
5,4113,1.0,0.1394,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Treatment with docetaxel,"[treatment, docetaxel]"
6,4114,1.0,0.1500,"event, treatment, group, advers, studi, popul, treat, summari, week, report","Treatments ever used at relapse, at the discretion of the treating clinician","[treatment, relaps, discret, treat, clinician]"
7,4115,1.0,0.2112,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Worst adverse event (grade) reported over entire time on trial,"[worst, advers, event, grade, report, entir, time, trial]"


In [55]:
df_topic_sents_keywords_mallet.shape

(32049, 6)

In [56]:
# df_topic_sents_keywords_tfidf = format_topics_sentences(model=tfidf_model, \
#                                                   corpus=processed_docs, \
#                                                   texts=dictionary_made_by, \
#                                                  model_type='tfidf')
# df_topic_sents_keywords_tfidf.head(max_coherence_topic_num_tfidf)

In [57]:
# df_topic_sents_keywords_on = format_topics_sentences(model=on_model, \
#                                                   corpus=processed_docs, \
#                                                   texts=dictionary_made_by, \
#                                                  model_type='online')
# df_topic_sents_keywords_on.head(max_coherence_topic_num_on)

In [58]:
print(df_topic_sents_keywords_mallet.isna().any())
print('=================')
# print(df_topic_sents_keywords_tfidf.isna().any())
# print('=================')
# print(df_topic_sents_keywords_on.isna().any())

id                   False
Dominant_Topic       False
Perc_Contribution    False
Topic_Keywords       False
Origin_Text          False
Text_Vec             False
dtype: bool


#### 각 토픽별로 가장 대표적인 문서 찾기

In [59]:
def get_rep_sents(keywords, model_type):
    sent_topics_sorted_df_mallet = pd.DataFrame()
    
    sent_topics_groupby = keywords.groupby('Dominant_Topic')

    keywords_top = pd.DataFrame()
    for i, grp in sent_topics_groupby:
        keywords_top = pd.concat([keywords_top, \
                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(50)], \
                            axis=0)
        keywords = pd.concat([keywords, \
                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(50)], \
                            axis=0)

    keywords_top.reset_index(drop=True, inplace=True)
    keywords_top.columns = ['id', 'Topic_Num', 'Topic_Perc_Contribu', 'Topic_Keywords', 'Origin_Text', 'Text']
    keywords_top.to_csv(directory+'/'+model_type+'_top_sen.tsv', sep='\t')
    
    keywords.reset_index(drop=True, inplace=True)
    keywords.columns = ['id', 'Topic_Num', 'Topic_Perc_Contribu', 'Topic_Keywords', 'Origin_Text', 'Text']
    
    return keywords
    
df_topic_sents_mallet = get_rep_sents(df_topic_sents_keywords_mallet, 'mallet')
# df_topic_sents_tfidf = get_rep_sents(df_topic_sents_keywords_tfidf, 'tfidf')
# df_topic_sents_on = get_rep_sents(df_topic_sents_keywords_on, 'online')

In [60]:
df_topic_sents_mallet.shape

(32449, 6)

In [61]:
df_topic_sents_mallet.drop_duplicates(subset=['id'], inplace=True)

In [62]:
df_topic_sents_mallet.shape

(32049, 6)

In [64]:
from db_conn import get_connection

# conn = get_connection()
# conn.set_client_encoding('UTF8')


def update_topic_num(keyword_df):
    _cur = conn.cursor()
    update_sql = """UPDATE article_tables
                    SET topic_num_title = %s,
                    topic_num_title_perc = %s
                    WHERE
                     id=%s"""
    
    for row in keyword_df.iterrows():
        _id = row[1]['id']
        topic_num = row[1]['Topic_Num']
#         print(topic_num)
        perc = row[1]['Topic_Perc_Contribu']
#         print(perc)
        _cur.execute(update_sql, (topic_num, perc, _id, ))
        conn.commit()

In [65]:
# 전체 문서 찾기

sent_topics_sorted_df_mallet = pd.DataFrame()

sent_topics_groupby = df_topic_sents_keywords_mallet.groupby('Dominant_Topic')

for i, grp in sent_topics_groupby:
    keywords = pd.concat([df_topic_sents_keywords_mallet, \
                         grp.sort_values(['Perc_Contribution'], ascending=[0])], \
                        axis=0)

keywords.reset_index(drop=True, inplace=True)
keywords.columns = ['id', 'Topic_Num', 'Topic_Perc_Contribu', 'Topic_Keywords', 'Origin_Text', 'Text']

keywords.head()

,id,Topic_Num,Topic_Perc_Contribu,Topic_Keywords,Origin_Text,Text
0,4106,1.0,0.1394,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Analysis of efficacy,"[analysi, efficaci]"
1,4107,2.0,0.1473,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,"[comparison, postop, ca, level, surviv, espac, conoko, jaspac, trial]"
2,4108,0.0,0.1368,"patient, data, comparison, score, respons, time, accord, test, status, assess",Pattern of disease relapse,"[pattern, diseas, relaps]"
3,4109,1.0,0.1898,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]"
4,4112,2.0,0.1515,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Treatment with zoledronic acid,"[treatment, zoledron, acid]"


In [66]:
# update 할때만 주석 해제
# update_topic_num(keywords)

In [67]:
df_topic_sents_keywords_mallet.head(5)

,id,Dominant_Topic,Perc_Contribution,Topic_Keywords,Origin_Text,Text_Vec
0,4106,1.0,0.1394,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Analysis of efficacy,"[analysi, efficaci]"
1,4107,2.0,0.1473,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,"[comparison, postop, ca, level, surviv, espac, conoko, jaspac, trial]"
2,4108,0.0,0.1368,"patient, data, comparison, score, respons, time, accord, test, status, assess",Pattern of disease relapse,"[pattern, diseas, relaps]"
3,4109,1.0,0.1898,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]"
4,4112,2.0,0.1515,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Treatment with zoledronic acid,"[treatment, zoledron, acid]"


In [68]:
df_topic_sents_mallet.head(5)

,id,Topic_Num,Topic_Perc_Contribu,Topic_Keywords,Origin_Text,Text
0,4106,1.0,0.1394,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Analysis of efficacy,"[analysi, efficaci]"
1,4107,2.0,0.1473,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,"[comparison, postop, ca, level, surviv, espac, conoko, jaspac, trial]"
2,4108,0.0,0.1368,"patient, data, comparison, score, respons, time, accord, test, status, assess",Pattern of disease relapse,"[pattern, diseas, relaps]"
3,4109,1.0,0.1898,"event, treatment, group, advers, studi, popul, treat, summari, week, report",Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,"[grade, advers, event, gemcitabin, gemcitabin, plus, capecitabin]"
4,4112,2.0,0.1515,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control",Treatment with zoledronic acid,"[treatment, zoledron, acid]"


In [69]:
df_topic_sents_keywords_mallet.to_csv('./mallet_results.tsv', sep='\t')

In [70]:
df_topic_sents_keywords_mallet.shape

(32049, 6)

#### 문서 전체적인 토픽 분포

In [71]:
topic_counts = df_topic_sents_mallet['Topic_Num'].value_counts()
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

topic_num_keywords = df_topic_sents_mallet[['Topic_Num', 'Topic_Keywords']]

df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)

In [72]:
topic_num_keywords.drop_duplicates(inplace=True)
topic_num_keywords

,Topic_Num,Topic_Keywords
0,1.0,"event, treatment, group, advers, studi, popul, treat, summari, week, report"
1,2.0,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control"
2,0.0,"patient, data, comparison, score, respons, time, accord, test, status, assess"
10,6.0,"effect, number, trial, estim, type, intervent, case, compar, health, control"
13,5.0,"year, age, mortal, relat, ratio, rat, incid, adjust, death, caus"
21,7.0,"outcom, clinic, patient, follow, month, day, hospit, primari, procedur, secondari"
38,4.0,"diseas, result, coronari, myocardi, valu, arteri, angiograph, infarct, frequenc, find"
50,3.0,"analysi, risk, associ, multivari, factor, regress, model, predictor, variabl, cardiac"


In [73]:
df_dominant_topics.reset_index(level=0, inplace=True)
df_dominant_topics.columns = ['Topic_Num', 'Num_Documents', 'Perc_Documents']
df_dominant_topics = df_dominant_topics.merge(topic_num_keywords, on='Topic_Num')
df_dominant_topics = df_dominant_topics.sort_values('Topic_Num')

df_dominant_topics.reset_index(drop = True, inplace = True)
df_dominant_topics.to_html(directory+'/mallet_distribution.html', index=False)
df_dominant_topics.head(max_coherence_topic_num)

,Topic_Num,Num_Documents,Perc_Documents,Topic_Keywords
0,0.0,4207,0.1313,"patient, data, comparison, score, respons, time, accord, test, status, assess"
1,1.0,4892,0.1526,"event, treatment, group, advers, studi, popul, treat, summari, week, report"
2,2.0,4350,0.1357,"chang, heart, paramet, function, hemodynam, measur, level, blood, rate, control"
3,3.0,4117,0.1285,"analysi, risk, associ, multivari, factor, regress, model, predictor, variabl, cardiac"
4,4.0,3769,0.1176,"diseas, result, coronari, myocardi, valu, arteri, angiograph, infarct, frequenc, find"
5,5.0,3416,0.1066,"year, age, mortal, relat, ratio, rat, incid, adjust, death, caus"
6,6.0,3175,0.0991,"effect, number, trial, estim, type, intervent, case, compar, health, control"
7,7.0,4123,0.1286,"outcom, clinic, patient, follow, month, day, hospit, primari, procedur, secondari"


In [74]:
# topic_counts = df_topic_sents_tfidf['Topic_Num'].value_counts()
# topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# topic_num_keywords = df_topic_sents_tfidf[['Topic_Num', 'Topic_Keywords']]

# df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)
# df_dominant_topics.reset_index(level=0, inplace=True)
# df_dominant_topics.columns = ['Topic_Num', 'Num_Documents', 'Perc_Documents']
# df_dominant_topics = df_dominant_topics.merge(topic_num_keywords.drop_duplicates(), on='Topic_Num')
# df_dominant_topics = df_dominant_topics.sort_values('Topic_Num')

# df_dominant_topics.reset_index(drop = True, inplace = True)
# df_dominant_topics.to_html(directory+'/tfidf_distribution.html', index=False)
# df_dominant_topics.head(max_coherence_topic_num)

In [75]:
# topic_counts = df_topic_sents_on['Topic_Num'].value_counts()
# topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# topic_num_keywords = df_topic_sents_on[['Topic_Num', 'Topic_Keywords']]

# df_dominant_topics = pd.concat([topic_counts, topic_contribution], axis=1)
# df_dominant_topics.reset_index(level=0, inplace=True)
# df_dominant_topics.columns = ['Topic_Num', 'Num_Documents', 'Perc_Documents']
# df_dominant_topics = df_dominant_topics.merge(topic_num_keywords.drop_duplicates(), on='Topic_Num')
# df_dominant_topics = df_dominant_topics.sort_values('Topic_Num')

# df_dominant_topics.reset_index(drop = True, inplace = True)
# df_dominant_topics.to_html(directory+'/on_distribution.html', index=False)
# df_dominant_topics.head(max_coherence_topic_num)

In [76]:
theta.shape

(32042, 8)

In [77]:
import pickle
import gensim    

# model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(mallet_model)
output = open('../data/output/lda.pkl', 'wb')
# pickle.dump({'topic_model': model, 
#               'corpus': bow_corpus, 
#               'dictionary': dictionary
#             }, output)
pickle.dump({
            'topic_term_dists':phi,
            'doc_topic_dists':theta,
            'doc_lengths':list(docs['doc_length']),
            'vocab':list(vocab['type']),
            'term_frequency':list(vocab['term_freq'])
        }, output)
output.close()

# topic_term_dists = lda_results['topic_term_dists']
# doc_topic_dists = lda_results['doc_topic_dists']
# doc_lengths = lda_results['doc_lengths']
# vocab = lda_results['vocab']
# term_frequency = lda_results['term_frequency']

# saved_model['lda_result'] = {
#         'document_topic_counts':topic_term_dists,
#         'topic_word_counts':doc_topic_dists,
#         'topic_counts':doc_lengths,
#         'document_lengths':vocab,
#         'distinct_words':term_frequency
#         }

- https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

- https://ai.googleblog.com/2016/12/open-sourcing-embedding-projector-tool.html
- http://projector.tensorflow.org/
- http://nbviewer.jupyter.org/github/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb#loading

In [ ]:
processed_docs.title_y.to_csv('../data/output/origin_text.tsv', sep='\t')

In [ ]:
processed_docs.title_x.to_csv('../data/output/processed.tsv', sep='\t')